# データ・パッケージの読込

## パッケージ読み込み

In [ ]:
### 必要なパッケージ（ライブラリ）の読み込み ###
import pandas as pd             # データフレーム型変数を取り扱えるパッケージ"pandas"を読込み（以降"pd"と略記）
import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）
import numpy as np              # 数値演算のためのパッケージ"numpy"を読込み（以降"np"と略記）

## 参考：描画設定
※重なりなどの描画崩れや、文字化けなど、描画がうまくいかない場合

In [ ]:
### 以下、描画の細かいデザイン設定 ###

# 出力されるグラフ画像の解像度を上げる
%config InlineBackend.figure_formats = {'png', 'retina'}

# グラフのサイズ指定
# plt.rcParams['figure.figsize'] = 10, 5

# 文字化け対策にフォント指定（Win/Macなどの環境に依らず、横断的に設定）
plt.rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao',
                                   'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

# フォントサイズ一括指定
# plt.rcParams['font.size'] = 14

## データ読み込み

In [ ]:
# データの読み込み
# 演習03_都道府県住民データ(2013年).xlsx
df = #各自入力

In [ ]:
df

## データの理解（観察）

### 欠損値の確認

In [ ]:
### 欠損値の確認 ###
df.isnull().sum()

### 散布図行列

In [ ]:
# 散布図行列による傾向確認
sns.pairplot(df) 
#sns.set(font='Yu Mincho') #日本語表示うまくいかない場合の予備

### 相関行列の確認

In [ ]:
### 相関行列による変数間の関係性確認 ###
df.corr(numeric_only=True)  # 相関行列の算出

In [ ]:
### 相関行列のお化粧（ヒートマップ） ###

# 描画の設定
colormap = plt.cm.RdBu_r                #カラーマップの設定 (RdBu:赤〜青 ⇔ RdBu_r:青〜赤)
plt.rcParams['figure.figsize'] = 10, 5  # Figureサイズの指定

# 相関行列の算出
corr = df.corr() #数値以外も含まれている場合には、numeric_only=True を引数に指定

# ヒートマップの描画
sns.heatmap(corr,linewidths=0.1, linecolor='white', #相関行列df.corr()を引数としヒートマップ作成	
            vmax=1.0, vmin=-1.0, cmap=colormap,     #最大値、最小値、カラーマップの指定 
            annot=True, fmt='.2f',                  #各セル内に表示する数値の設定
            # mask=np.triu(corr) #相関行列の右上半分を隠したい場合はこのコメントを解除
           )

            # linewidths/linecolor: 格子線の太さ/色
            # cmap:                 カラーマップの指定
            # vmax/vmin:            最大値/最小値
            # annot:                各要素への数値表示
            # annot_kws={'fontsize':8}: 各要素内に表示する数値の文字サイズを指定
            # fmt='.2f':            各要素内に表示する数値の小数点桁数を指定（'.2f'で小数第二位まで）
            # mask:                 可視化から除外する対象


# k-means法 / k-means++法

## モデルの構築

### 説明変数のセット

In [ ]:
# 説明変数のセット
X = df.drop(columns=['都道府県'])

### データの標準化

In [ ]:
from sklearn import preprocessing      # 機械学習パッケージscikit-learnから、データ前処理モジュールの読込

In [ ]:
### 標準化 ###
# 説明変数の標準化（Zスコア）
X = preprocessing.scale(X)

### k-meansクラスタリングモデルの構築

In [ ]:
from sklearn.cluster import KMeans # 機械学習パッケージに含まれたKMeansモジュール"KMeans"を読込

# クラスタ数の設定
n = 5

# クラスタリング
model = KMeans( n_clusters=n, random_state=0, init='k-means++')  #クラスタリングの実行準備
                                                # オプションの説明：
                                                #   n_clustersでクラスタ数指定
                                                #   random_state=0でモデルの再現性を保持（乱数のシードを固定）
                                                #   init='random'で初期値の置き方をランダムに。'k-means++'指定でk-means++法による最適な初期値配置
model.fit(X) #Xを説明変数としてクラスタリング実行

### 元データへのクラスタ番号紐付け

In [ ]:
### 元のdataframeにクラスタ番号を追加 ###
df['cluster']=model.labels_

In [ ]:
df #紐付け結果確認

## モデルの評価

### クラスタリング結果の可視化

In [ ]:
### 散布図行列でクラスタごとに色分けして可視化 ###
sns.pairplot( df, hue='cluster')

### クラスタリング結果の可視化

In [ ]:
### 特定の2変数で、クラスタごとに色分けして可視化＋クラスタ重心の可視化 ###

#  特定の2変数を選択
dfx = df['世帯の年収']
dfy = df['所要資金額']

#　特定の2変数でクラスタごとに色分け可視化
sns.scatterplot( dfx, dfy, hue=df['cluster'], palette=sns.color_palette(n_colors=n)  )
    #hue: 色分け対象
    #palette: 色パターン
    #クラスタごとにマーカー形状も変えたい場合は、style=df['cluster'] をオプションに加える

# # クラスタ重心 (mode.cluster_centers_) の可視化 ※model.cluster_centers_[:,0]で0番目の説明変数の重心座標
# sns.scatterplot( model.cluster_centers_[:,1], model.cluster_centers_[:,2], marker="*", s=300, color="purple" ) 

# 散布図上に、自動車メーカーの名前をプロット
for i, company in enumerate(df['都道府県']):
    plt.annotate(company, (dfx.values[i], dfy.values[i]) )

In [ ]:
### 特定の3変数で、クラスタごとに色分けして可視化（3次元プロット）」 ###

from mpl_toolkits.mplot3d.axes3d import Axes3D  #3次元プロットするためのモジュール

#  特定の3変数を選択
dfx = df['世帯の年収']
dfy = df['所要資金額']
dfz = df['住宅面積']

# グラフを手動で回転させるための設定
%matplotlib notebook

# 3次元グラフの枠を作る
fig = plt.figure()    #枠作成
ax = Axes3D(fig)      #3次元の軸を用意

# データをプロット
colors = {0:'blue', 1:'Orange', 2:'green', 3:'red', 4:'purple', 5:'pink', 6:'black'}
ax.scatter( dfx, dfy, dfz, s=50, c=df['cluster'].apply(lambda x: colors[x]) )
  # c=df['cluster']でクラスターごとに色を変更
  # s=50でプロットサイズ指定
  # alpha=1 をオプションに入れると透過性無し
    
# 軸ラベルの表示
ax.set_xlabel( dfx.name )
ax.set_ylabel( dfy.name )
ax.set_zlabel( dfz.name )

#fig.colorbar(p) # カラーバーを表示
 
plt.show()

In [ ]:
# グラフをインライン表示に戻す（3次元プロット後は、必ず実行しておくこと！）
# ※"%matplotlib notebook"の表示設定のままだと、3Dグラフ以外はうまく表示されないので元に戻す

%matplotlib inline


### 最適なクラスタ数の確認（Elbow法）

In [ ]:
SSE = []

# クラスタ数1~10まで振ってクラスタ内誤差平方和 (SSE) を計算

for i in range(1,11):                # 最後の11は含まれない start ≦ i ＜ stop なので要注意
    km = KMeans(n_clusters=i,
                init='k-means++',     # 初期値の置き方：k-means++ or random or 任意の数値配列
                n_init=10,            # 初期値選択において、異なる乱数のシードで初期の重心を選ぶ処理の実行回数。(デフォルト値: 10)
                max_iter=300,         # 繰り返し回数の最大値 (デフォルト値: 300)
                random_state=0        # 乱数シードの固定（再現性を担保）
               )
    km.fit(X)                         # クラスタリングの計算を実行
    SSE.append(km.inertia_)   # km.fitするとkm.inertia_ (クラスタ内誤差平方和:SSE) が得られる

# 得られた結果の可視化
plt.plot(range(1,11), SSE, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

### (参考)シルエット図によるクラスタ評価

In [ ]:

from sklearn.metrics import silhouette_samples
from matplotlib import cm

y_km = model.fit_predict(X)
cluster_labels = np.unique(y_km)       # y_kmの要素の中で重複を無くす
n_clusters=cluster_labels.shape[0]     # 配列の長さ=クラスタ数

# シルエット係数を計算
silhouette_vals = silhouette_samples(X,y_km,metric='euclidean')  # サンプルデータ, クラスター番号、ユークリッド距離でシルエット係数計算
y_ax_lower, y_ax_upper= 0,0
yticks = []

for i,c in enumerate(cluster_labels):
        c_silhouette_vals = silhouette_vals[y_km==c]      # cluster_labelsには 0,1,2 が格納
        c_silhouette_vals.sort()
        y_ax_upper += len(c_silhouette_vals)              # サンプルの個数をクラスターごとに足し上げてy軸の最大値を決定
        color = cm.jet(float(i)/n_clusters)               # 色の値を作る
        plt.barh(range(y_ax_lower,y_ax_upper),            # 水平の棒グラフのを描画（底辺の範囲を指定）
                         c_silhouette_vals,               # 棒の幅（1サンプルを表す）
                         height=1.0,                      # 棒の高さ
                         edgecolor='none',                # 棒の端の色
                         color=color)                     # 棒の色
        yticks.append((y_ax_lower+y_ax_upper)/2)          # クラスタラベルの表示位置を追加
        y_ax_lower += len(c_silhouette_vals)              # 底辺の値に棒の幅を追加

silhouette_avg = np.mean(silhouette_vals)                 # シルエット係数の平均値
plt.axvline(silhouette_avg,color="red",linestyle="--")    # 係数の平均値に破線を引く 
plt.yticks(yticks,cluster_labels + 1)                     # クラスタレベルを表示
plt.ylabel('Cluster')
plt.xlabel('silhouette coefficient')
plt.show()

# x-meansクラスタリングモデルの構築

## 高度なクラスタリング手法のパッケージインストール

In [ ]:
###  Anaconda Promptでのpyclusteringのパッケージインストール ###
# 初回のみ実行（※Macの場合はターミナル上からでOK）

# 下記コマンドをコピペして使用
# conda install -c conda-forge pyclustering

# ※どうしても上記でエラーが出てうまくいかない場合は、 pip install pyclustering を用いる
#.  ただし、conda install と pip install は相性が悪い部分もあり、公式でも非推奨

## モデルの構築

In [ ]:
import pyclustering
from pyclustering.cluster import xmeans  # pyclustringパッケージに含まれたxmeansモジュールを読込

# 説明変数のセット
X = df.drop(columns=['都道府県']).values  # pyclusteringの仕様で、dataframet型ではなくarray型でデータを渡す必要があるため型変換

# 初期状態をセット
initial_centers = xmeans.kmeans_plusplus_initializer(data=X, amount_centers=2).initialize()

# クラスタリング
xm = xmeans.xmeans(data=X, initial_centers=initial_centers)
xm.process()

# クラスタリング結果の取得
clusters = xm.get_clusters()

## モデルの評価

### クラスタ数

In [ ]:
# 分割クラスタ数の出力
print(len(clusters))

### グラフ出力

In [ ]:
# 2次元グラフ可視化
pyclustering.utils.draw_clusters(data=X[:,:2], clusters=clusters)

In [ ]:
# 3次元グラフ可視化
pyclustering.utils.draw_clusters(data=X[:,:3], clusters=clusters)

### （参考）ボロノイ図

In [ ]:
import scipy
from scipy.spatial import voronoi_plot_2d, Voronoi

%matplotlib inline

plt.style.use('ggplot')

_ = plt.scatter(x=X[:, 0], y=X[:, 1])

In [ ]:
vor = Voronoi(points=X[:,:2])
_ = voronoi_plot_2d(vor=vor)

# DBSCAN

## モデル構築

### 説明変数のセット

In [ ]:
# 説明変数のセット
X = df.drop(columns=['都道府県'])

### データの標準化

In [ ]:
from sklearn import preprocessing      # 機械学習パッケージscikit-learnから、データ前処理モジュールの読込

In [ ]:
### 標準化 ###
# 説明変数の標準化（Zスコア）
X = preprocessing.scale(X)

In [ ]:
from sklearn.cluster import DBSCAN # DBSCANモジュールのインポート

# 説明変数のセット
X = df.drop(columns=['都道府県'])

# モデル構築準備
model = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
        # 各種ハイパーパラメータ
        #   eps: 隣接点と見なす2点間の最大距離
        #   min_samples: ボーダー点の最小個数
        #   metric: 距離の計算方法

# モデル構築
model.fit_predict(X)

### 元データへのクラスタ番号紐付け

In [ ]:
### 元のdataframeにクラスタ番号を追加 ###
df['cluster']=model.labels_

In [ ]:
df #紐付け結果確認

## モデルの評価

### クラスタリング結果の可視化

In [ ]:
### 散布図行列でクラスタごとに色分けして可視化 ###
sns.pairplot( df, hue='cluster')

### クラスタリング結果の可視化

In [ ]:
### 特定の2変数で、クラスタごとに色分けして可視化＋クラスタ重心の可視化 ###

#  特定の2変数を選択
dfx = df['世帯の年収']
dfy = df['所要資金額']

#　特定の2変数でクラスタごとに色分け可視化
sns.scatterplot( dfx, dfy, hue=df['cluster'], palette=sns.color_palette(n_colors=n)  )
    #hue: 色分け対象
    #palette: 色パターン
    #クラスタごとにマーカー形状も変えたい場合は、style=df['cluster'] をオプションに加える

# # クラスタ重心 (mode.cluster_centers_) の可視化 ※model.cluster_centers_[:,0]で0番目の説明変数の重心座標
# sns.scatterplot( model.cluster_centers_[:,1], model.cluster_centers_[:,2], marker="*", s=300, color="purple" ) 

# 散布図上に、自動車メーカーの名前をプロット
for i, company in enumerate(df['都道府県']):
    plt.annotate(company, (dfx.values[i], dfy.values[i]) )

In [ ]:
### 特定の3変数で、クラスタごとに色分けして可視化（3次元プロット）」 ###

from mpl_toolkits.mplot3d.axes3d import Axes3D  #3次元プロットするためのモジュール

#  特定の3変数を選択
dfx = df['世帯の年収']
dfy = df['所要資金額']
dfz = df['住宅面積']

# グラフを手動で回転させるための設定
%matplotlib notebook

# 3次元グラフの枠を作る
fig = plt.figure()    #枠作成
ax = Axes3D(fig)      #3次元の軸を用意

# データをプロット
colors = {0:'blue', 1:'Orange', 2:'green', 3:'red', 4:'purple', 5:'pink', 6:'black'}
ax.scatter( dfx, dfy, dfz, s=50, c=df['cluster'].apply(lambda x: colors[x]) )
  # c=df['cluster']でクラスターごとに色を変更
  # s=50でプロットサイズ指定
  # alpha=1 をオプションに入れると透過性無し
    
# 軸ラベルの表示
ax.set_xlabel( dfx.name )
ax.set_ylabel( dfy.name )
ax.set_zlabel( dfz.name )

#fig.colorbar(p) # カラーバーを表示
 
plt.show()

In [ ]:
# グラフをインライン表示に戻す（3次元プロット後は、必ず実行しておくこと！）
# ※"%matplotlib notebook"の表示設定のままだと、3Dグラフ以外はうまく表示されないので元に戻す

%matplotlib inline


# 混合ガウスモデル (GMM)

## モデルの構築

### 説明変数のセット

In [ ]:
# 説明変数のセット
X = df.drop(columns=['都道府県'])

### データの標準化
※GMMの場合は、標準化は必ずしも不要

In [ ]:
# from sklearn import preprocessing      # 機械学習パッケージscikit-learnから、データ前処理モジュールの読込

In [ ]:
# ### 標準化 ###
# # 説明変数の標準化（Zスコア）
# X = preprocessing.scale(X)

### モデルの構築

In [ ]:
from sklearn.mixture import GaussianMixture # 機械学習パッケージに含まれたKMeansモジュール"GaussianMixture"を読込

# クラスタ数の設定
n = 3

# クラスタリング
model = GaussianMixture(n_components=n)

model.fit(X) #Xを説明変数としてクラスタリング実行

### ハードクラスタリング結果のクラスタ番号紐付け
※確率最大値のクラスタ番号を採用した結果

In [ ]:
### 元のdataframeにクラスタ番号を追加 ###
df['cluster'] =  model.predict(X)

In [ ]:
df #紐付け結果確認

## モデルの評価

### クラスタリング結果の可視化

In [ ]:
### 散布図行列でクラスタごとに色分けして可視化 ###
sns.pairplot( df, hue='cluster')

### クラスタリング結果の可視化

In [ ]:
### 特定の2変数で、クラスタごとに色分けして可視化＋クラスタ重心の可視化 ###

#  特定の2変数を選択
dfx = df['世帯の年収']
dfy = df['所要資金額']

#　特定の2変数でクラスタごとに色分け可視化
sns.scatterplot( dfx, dfy, hue=df['cluster'], palette=sns.color_palette(n_colors=n)  )
    #hue: 色分け対象
    #palette: 色パターン
    #クラスタごとにマーカー形状も変えたい場合は、style=df['cluster'] をオプションに加える

# # クラスタ重心 (mode.cluster_centers_) の可視化 ※model.cluster_centers_[:,0]で0番目の説明変数の重心座標
# sns.scatterplot( model.cluster_centers_[:,1], model.cluster_centers_[:,2], marker="*", s=300, color="purple" ) 

# 散布図上に、自動車メーカーの名前をプロット
for i, company in enumerate(df['都道府県']):
    plt.annotate(company, (dfx.values[i], dfy.values[i]) )

In [ ]:
### 特定の3変数で、クラスタごとに色分けして可視化（3次元プロット）」 ###

from mpl_toolkits.mplot3d.axes3d import Axes3D  #3次元プロットするためのモジュール

#  特定の3変数を選択
dfx = df['世帯の年収']
dfy = df['所要資金額']
dfz = df['住宅面積']

# グラフを手動で回転させるための設定
%matplotlib notebook

# 3次元グラフの枠を作る
fig = plt.figure()    #枠作成
ax = Axes3D(fig)      #3次元の軸を用意

# データをプロット
colors = {0:'blue', 1:'Orange', 2:'green', 3:'red', 4:'purple', 5:'pink', 6:'black'}
ax.scatter( dfx, dfy, dfz, s=50, c=df['cluster'].apply(lambda x: colors[x]) )
  # c=df['cluster']でクラスターごとに色を変更
  # s=50でプロットサイズ指定
  # alpha=1 をオプションに入れると透過性無し
    
# 軸ラベルの表示
ax.set_xlabel( dfx.name )
ax.set_ylabel( dfy.name )
ax.set_zlabel( dfz.name )

#fig.colorbar(p) # カラーバーを表示
 
plt.show()

### ソフトクラスタリング結果
※各クラスタへの所属確率を出力

In [ ]:
### 構築したGMMから、各クラスタへの所属確率を抽出 ###
cluster_prob = model.predict_proba(X)
print(cluster_prob)

In [ ]:
#  特定の2変数を選択して、可視化
dfx = df['世帯の年収']
dfy = df['所要資金額']
plt.scatter(dfx, dfy, c=[target_to_color(t) for t in model.predict_proba(X)])

# 凝集型階層的クラスタリング

# モデルの構築

### 説明変数のセット

In [ ]:
# 説明変数のセット
X = df.drop(columns=['都道府県'])

### データの標準化

In [ ]:
from sklearn import preprocessing      # 機械学習パッケージscikit-learnから、データ前処理モジュールの読込

In [ ]:
### 標準化 ###
# 説明変数の標準化（Zスコア）
X = preprocessing.scale(X)

### クラスタリング

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

# クラスタリング
linkage_result = linkage( X, method='ward', metric='euclidean')
                # metric: 距離の計量方法（euclidean: ユークリッド距離）
                # method: 手法 ("average", "centroid", "complete", "median", "single", "ward", "weighted")

# デンドログラムの描画
dendrogram( linkage_result, labels=df['都道府県'].values )

plt.rcParams['figure.figsize'] = 20,5  # グラフサイズを大きく
plt.tick_params(labelsize=18)          # フォントサイズを大きく
plt.show()

# 各種クラスタリング手法の比較

In [ ]:
# 参考サイト：Comparing different clustering algorithms on toy datasets — scikit-learn 0.22 documentation
# - https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html
        
print(__doc__)

import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

import pyclustering                        # 高度なクラスタリング手法のpyclusteringパッケージをインポート
from pyclustering.cluster import xmeans    # pyclustering.cluster配下の xmeans手法をインポート
                                           #（この1行が無いと毎回 "pyclustering.cluster.xmeans" と呼び出す必要がある）

np.random.seed(0)

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

datasets = [
    (blobs, {}),
    (varied, {'eps': .18, 'n_neighbors': 2,
          'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
         'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (noisy_circles, {'damping': .77, 'preference': -240,
                 'quantile': .2, 'n_clusters': 2,
                 'min_samples': 20, 'xi': 0.25}),
    (no_structure, {})]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=params['eps'])
#     optics = cluster.OPTICS(min_samples=params['min_samples'],
#                             xi=params['xi'],
#                             min_cluster_size=params['min_cluster_size'])
#     affinity_propagation = cluster.AffinityPropagation(
#         damping=params['damping'], preference=params['preference'])
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
#     birch = cluster.Birch(n_clusters=params['n_clusters'])
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('KMeans (MiniBatch)', two_means),
#         ('AffinityPropagation', affinity_propagation),
        ('GaussianMixture', gmm),
        ('DBSCAN', dbscan),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ward', ward),
        ('AgglomerativeClustering', average_linkage)
#         ('OPTICS', optics),
#         ('Birch', birch),
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()